<a href="https://colab.research.google.com/github/viviantlam/BME3053C-Spring-2025/blob/main/lessons-solved/05_Advanced_DataFrame_Operations_02_19_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BME3053C - Computer Applications for BME

<br/>

<h1 align="center">Advanced DataFrame Operations</h1>

---

<center><h2>Lesson: 05</h2></center>

<br/>

<center><img src='https://github.com/snsie/aicc24/blob/main/graphics/pandas_logo.png?raw=1' alt='The Pandas logo' align='center' width=200></center>

<br/>

### Original Lesson Link: [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/uf-bme/BME3053C-Spring-2025/blob/main/lessons/05_Advanced_DataFrame_Operations.ipynb)

### **Covered Topics**

1. **Indexing and Iteration**

   - Different ways to access data in DataFrames
   - Efficient methods for iterating through rows and columns

2. **Combining DataFrames**

   - Merging DataFrames
   - Concatenating DataFrames

3. **Data Cleaning**
   - Handling missing values and duplicates
   - Data type conversion


#### Import Pandas


In [3]:
import pandas as pd

## Indexing and Iteration

### Accessing Data in DataFrames

- Use `.iloc[]` for integer position-based indexing (e.g., df.iloc[0, 1] for first row, second column)
- Use `.loc[]` for label-based indexing (e.g., df.loc['row_label', 'column_name'])

- Example:
  - df.iloc[0, 1] gets value in first row, second column regardless of labels
  - df.loc['A', 'price'] gets value where index='A' and column='price'


In [4]:
sample_data = {
    'A': [1, 2, 3, 4, 5],
    'B': [10, 20, 30, 40, 50],
    'C': [100, 200, 300, 400, 500]
}
df = pd.DataFrame(sample_data)
df.head()

,A,B,C
0,1,10,100
1,2,20,200
2,3,30,300
3,4,40,400
4,5,50,500


In [ ]:
# Access first row using integer-based indexing
print('\nFirst row using iloc:')
df.iloc[0]


In [ ]:

# Access first 3 rows and first 3 columns using iloc
print('\nFirst 3 rows and first 3 columns using iloc:')
print(df.iloc[0:3, 0:3])


In [ ]:
# Access first row using label-based indexing
# Note: For this dataset, .loc[] with integer index acts similar to .iloc[]
# because the index is numeric and sequential starting from 0

print('First row using loc:')
print(df.loc[0])


In [ ]:
# Access first 3 rows and specific columns using loc
print('\nFirst 3 rows and selected columns using loc:')
print(df.loc[0:2, ['A', 'B']])

### When .loc with integer index differs from .iloc

1. **Non-sequential index**: If DataFrame index is not 0,1,2,...

   - `.loc[0]` looks for row with index label 0 (may not exist)
   - `.iloc[0]` gets first row regardless of its index

2. **Non-numeric index**: If DataFrame uses string/date indices

   - `.loc[]` won't work with integer input
   - `.iloc[]` still works with integer positions

3. **Gaps in numeric index**: If index like [0,2,5,...]
   - `.loc[1]` looks for index 1 (doesn't exist)
   - `.iloc[1]` gets second row (index 2)

Example showing difference with non-sequential index:


In [5]:

example_df = pd.DataFrame({'A': [1,2,3], 'B': [4,5,6]}, index=[10,20,30])
print("\nExample with non-sequential index:")
print("Using .loc[10]:")  # Gets row with index label 10
print(example_df.loc[10])
print("\nUsing .iloc[0]:")  # Gets first row regardless of index
print(example_df.iloc[0])


Example with non-sequential index:
Using .loc[10]:
A    1
B    4
Name: 10, dtype: int64

Using .iloc[0]:
A    1
B    4
Name: 10, dtype: int64


#### ✏️ **Exercise**

1. Create a DataFrame with the following data:
   - Columns: 'Name', 'Age', 'City'
   - Data: [['Alice', 25, 'New York'], ['Bob', 30, 'Boston'], ['Charlie', 35, 'Chicago']]
   - Index: [100, 200, 300]
2. Use .loc[] to:
   - Get the row with index 200
3. Use .iloc[] to:
   - Get the first row
   - Get the second and third rows with only the 'Name' and 'City' columns


In [6]:
data_frame = pd.DataFrame({'Name': ['Alice', 'Bob', 'Charlie'], 'Age': [25, 30, 35], 'City': ['New York', 'Boston', 'Chicago']}, index=[100, 200, 300])

print(data_frame.loc[200])
print(data_frame.iloc[0])
print(data_frame.iloc[1:3, [0, 2]])

Name       Bob
Age         30
City    Boston
Name: 200, dtype: object
Name       Alice
Age           25
City    New York
Name: 100, dtype: object
        Name     City
200      Bob   Boston
300  Charlie  Chicago


## Iterating Through Rows and Columns

#### Using .iterrows()

- Iterates through DataFrame rows as (index, Series) pairs
- Each row is returned as a pandas Series object
- Useful for row-wise operations
- Note: Can be slower than vectorized operations

#### Using .items()

- Iterates through DataFrame columns as (column_name, Series) pairs
- Each column is returned as a pandas Series object
- Useful for column-wise operations

#### Best Practices:

- Use vectorized operations when possible for better performance
- Only use iteration when necessary for custom row/column operations
- Consider using .apply() as an alternative to explicit loops


In [ ]:
for index, row in df.iterrows():
    print(f"Row {index}: {row['A']}")
    if index > 3:
        break


In [ ]:
for column_name, column_data in df.items():
    if column_name == 'A':  # Stop after mean radius column
        print(f"Column {column_name}: {column_data[0:5]}")  # Print first value in each column
        break


#### ✏️ **Exercise**

Create a loop using .items() to find the maximum value in each column of df.
Print the column name and its maximum value.

- Example output:

`A: 5`

`B: 50`

`C: 500`


In [ ]:
for column_name, column_data in df.items():
    print(f"{column_name}: {column_data.max()}")

A: 5
B: 50
C: 500


## Combining DataFrames

### Concatenating Datasets with pd.concat()

- Use `pd.concat()` to stack DataFrames vertically or horizontally
- Parameters:
  - `axis=0`: Stack vertically (default)
  - `axis=1`: Stack horizontally
  - `ignore_index`: Reset index after concatenation


In [7]:
# Create sample DataFrames
df1 = pd.DataFrame({
    'A': ['A0', 'A1', 'A2'],
    'B': ['B0', 'B1', 'B2']
})

df2 = pd.DataFrame({
    'A': ['A3', 'A4', 'A5'],
    'B': ['B3', 'B4', 'B5']
})

df3 = pd.DataFrame({
    'C': ['C0', 'C1', 'C2'],
    'D': ['D0', 'D1', 'D2']
})

print("DataFrame 1:")
print(df1)
print("\nDataFrame 2:")
print(df2)
print("\nDataFrame 3:")
print(df3)

# Vertical concatenation (axis=0)
vertical_concat = pd.concat([df1, df2])
print("\nVertical concatenation (axis=0):")
print(vertical_concat)

DataFrame 1:
    A   B
0  A0  B0
1  A1  B1
2  A2  B2

DataFrame 2:
    A   B
0  A3  B3
1  A4  B4
2  A5  B5

DataFrame 3:
    C   D
0  C0  D0
1  C1  D1
2  C2  D2

Vertical concatenation (axis=0):
    A   B
0  A0  B0
1  A1  B1
2  A2  B2
0  A3  B3
1  A4  B4
2  A5  B5


In [8]:
print("DataFrame 1:")
print(df1)
print("\nDataFrame 2:")
print(df2)
# Vertical concatenation with reset index
vertical_concat_reset = pd.concat([df1, df2], ignore_index=True)
print("\nVertical concatenation with reset index:")
print(vertical_concat_reset)

DataFrame 1:
    A   B
0  A0  B0
1  A1  B1
2  A2  B2

DataFrame 2:
    A   B
0  A3  B3
1  A4  B4
2  A5  B5

Vertical concatenation with reset index:
    A   B
0  A0  B0
1  A1  B1
2  A2  B2
3  A3  B3
4  A4  B4
5  A5  B5


In [9]:
print("DataFrame 1:")
print(df1)

print("\nDataFrame 3:")
print(df3)
# Horizontal concatenation (axis=1)
horizontal_concat = pd.concat([df1, df3], axis=1)
print("\nHorizontal concatenation (axis=1):")
print(horizontal_concat)


DataFrame 1:
    A   B
0  A0  B0
1  A1  B1
2  A2  B2

DataFrame 3:
    C   D
0  C0  D0
1  C1  D1
2  C2  D2

Horizontal concatenation (axis=1):
    A   B   C   D
0  A0  B0  C0  D0
1  A1  B1  C1  D1
2  A2  B2  C2  D2


### Merging Datasets with pd.merge()

- Use `pd.merge()` to combine DataFrames based on common columns/keys
- Similar to SQL JOIN operations
- Use `on`: to specify the column(s) to merge on


In [10]:
# Single key merge
df1 = pd.DataFrame({'key': ['A', 'B', 'C'], 'columnA': [1, 2, 3]})
df2 = pd.DataFrame({'key': ['A', 'B'], 'columnB': [3, 4]})
merged_single = pd.merge(df1, df2, on='key')

print("DataFrame 1:")
print(df1)

print("\nDataFrame 2:")
print(df2)

print("\nSingle key merge:")
print(merged_single)



DataFrame 1:
  key  columnA
0   A        1
1   B        2
2   C        3

DataFrame 2:
  key  columnB
0   A        3
1   B        4

Single key merge:
  key  columnA  columnB
0   A        1        3
1   B        2        4


In [11]:
# Multiple key merge
df3 = pd.DataFrame({'ColumnA': ['A', 'B', 'C', 'D'], 'ColumnB': [1, 2, 3, 4], 'ColumnX': [1, 2, 3, 4]})
df4 = pd.DataFrame({'ColumnA': ['B', 'C', 'D', 'E'], 'ColumnB': [2, 3, 1, 2], 'ColumnY': [4, 5, 6, 7]})
merged_multi = pd.merge(df3, df4, on=['ColumnA', 'ColumnB'])

print("DataFrame 1:")
print(df3)

print("\nDataFrame 2:")
print(df4)

print("Multiple key merge:")
print(merged_multi)
print()


DataFrame 1:
  ColumnA  ColumnB  ColumnX
0       A        1        1
1       B        2        2
2       C        3        3
3       D        4        4

DataFrame 2:
  ColumnA  ColumnB  ColumnY
0       B        2        4
1       C        3        5
2       D        1        6
3       E        2        7
Multiple key merge:
  ColumnA  ColumnB  ColumnX  ColumnY
0       B        2        2        4
1       C        3        3        5



#### ✏️ **Exercise**

Try to predict the output of the following operations:

1. What will happen when we merge these DataFrames?
   ```python
   df1 = pd.DataFrame({'key': ['A','B'], 'val': [1,2]})
   df2 = pd.DataFrame({'key': ['B','C'], 'val': [3,4]})
   pd.merge(df1, df2, on='key')
   ```
2. What will happen when we concatenate the same DataFrames?
   ```python
   pd.concat([df1, df2])
   ```
   Key differences to note:

- merge: Combines based on matching values in the 'key' column
- concat: Simply stacks the DataFrames, keeping all rows
  Check your predictions in the next code cell!


## Data Cleaning and Preprocessing

### Checking for Missing Values

- Use `.isna()` or `.isnull()` to check for missing values.
- Use `.notna()` or `.notnull()` to check for non-missing values.
- Use `.any()` to see if any values are missing in a DataFrame.
- Use `.sum()` to count the number of missing values in each column.


In [12]:
df_with_nan = pd.DataFrame({'A': [None, None, 3, 1], 'B': [4, None, None, 33]})
df_with_nan.iloc[0, 0] = pd.NA  # Introduce pd.NA to make .isna different from .isnull
print("DataFrame with NaN, None, and pd.NA values:")
print(df_with_nan)


DataFrame with NaN, None, and pd.NA values:
     A     B
0  NaN   4.0
1  NaN   NaN
2  3.0   NaN
3  1.0  33.0


In [13]:
print("\nCheck for missing values using .isna():")
print(df_with_nan.isna())



Check for missing values using .isna():
       A      B
0   True  False
1   True   True
2  False   True
3  False  False


isna


In [14]:
print("\nCheck for non-missing values using .notna():")
print(df_with_nan.notna())



Check for non-missing values using .notna():
       A      B
0  False   True
1  False  False
2   True  False
3   True   True


In [15]:
print("\nCheck if any values are missing in the DataFrame using .any():")
print(df_with_nan.isna().any())



Check if any values are missing in the DataFrame using .any():
A    True
B    True
dtype: bool


OPERATES LEFT TO RIGHT WHEN STACKING METHODS


In [16]:

print("\nCount the number of missing values in each column using .sum():")
print(df_with_nan.isna().sum())



Count the number of missing values in each column using .sum():
A    2
B    2
dtype: int64


### Handling Missing Values

- Use `.fillna()` to REPLACE missing values with a specified value.
- Use `.dropna()` to REMOVE rows or columns with missing values.


In [17]:
df_with_nan = pd.DataFrame({'A': [1, None, 3], 'B': [4, 5, None]})
filled_df = df_with_nan.fillna(0) # FILL MISSING VALUES W ZEROS
print(filled_df)


     A    B
0  1.0  4.0
1  0.0  5.0
2  3.0  0.0


In [18]:
df_with_nan = pd.DataFrame({'A': [1, None, 3], 'B': [4, 5, None]})
dropped_df = df_with_nan.dropna() # DROP ROWS W MISSING VALUES
print(dropped_df)


     A    B
0  1.0  4.0


ARTIFICIALLY GENERATED DATA CAN BIAS MODELS
need to check if others in field are replacing missing values w zero to standardize data!
usually safer to remove than replace

### Data Type Conversion

- Use `.astype()` to convert data types.


In [19]:
df = pd.DataFrame({'A': [1, 2, 3], 'B': [4.0, 5.0, 6.0]})
print('before conversion:', df.dtypes)
df['A'] = df['A'].astype(float)
print('after conversion:', df.dtypes)

before conversion: A      int64
B    float64
dtype: object
after conversion: A    float64
B    float64
dtype: object


#### ✏️ Exercise: DataFrame Treasure Hunt

In this exercise, you'll be a data detective on a treasure hunt.
Your goal is to clean and preprocess the given DataFrame to uncover hidden treasures (valuable insights).

##### Step 1: Create the DataFrame

Create a DataFrame with the following data:

```python
data = {
    'Name': ['Alice', 'Bob', None, 'David', 'Eva'],
    'Age': [25, None, 35, 45, None],
    'Score': ['85.5', '90.0', None, '088.0', '92.0']
}
```

##### Step 2: Print the person 'Name' with the highest score who isn't missing values in any column

**Hint: You can use the [sort_values](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sort_values.html) method to sort the DataFrame by the 'Score' column.**

When sorting values in a DataFrame, the original indices are preserved.
If you want to reset the indices after sorting, you can use the reset_index method.

For example:

```python
sorted_df = df.sort_values(by='Column_Name', ascending=False).reset_index(drop=True)
```


In [23]:
data = {
    'Name': ['Alice', 'Bob', None, 'David', 'Eva'],
    'Age': [25, None, 35, 45, None],
    'Score': ['85.5', '90.0', None, '088.0', '92.0']
}

df = pd.DataFrame(data)
df['Score'] = df['Score'].astype(float)

dropped_df = df.dropna()

sorted_df = dropped_df.sort_values(by='Score', ascending=False).reset_index(drop=True)
  # sorts dataframe into DESCENDING ORDER of score values
  # values in other columns are also moved along when sorted by score

print(sorted_df.iloc[0, 0])
  # prints FIRST NAME (row, col) bc row0 = name, col0 = eva
  # eva name first bc highest score and values sorted in descending order

David
